In [58]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re

In [64]:
# source https://gist.github.com/dobrosketchkun/f14e1ab9ae817b00b28251f11786fbcf
import requests


class CrossRefClient(object):

    def __init__(self, accept='text/x-bibliography; style=apa', timeout=3):
        """
        # Defaults to APA biblio style

        # Usage:
        s = CrossRefClient()
        print s.doi2apa("10.1038/nature10414")
        """
        self.headers = {'accept': accept}
        self.timeout = timeout

    def query(self, doi, q={}):
        if doi.startswith("http://"):
            url = doi
        else:
            url = "http://dx.doi.org/" + doi
        r = requests.get(url, headers = self.headers)
        return r

    def doi2apa(self, doi):
        self.headers['accept'] = 'text/x-bibliography; style=apa'
        return self.query(doi).text

    def doi2turtle(self, doi):
        self.headers['accept'] = 'text/turtle'
        return self.query(doi).text

    def doi2json(self, doi):
        self.headers['accept'] = 'application/vnd.citationstyles.csl+json'
        return self.query(doi).json()


def names(ref):
    name = []
    for _item in ref['author']:
        given_in = _item['given'].split(' ')
        given = ''.join([_name[0] + '.' for _name in given_in])
        name.append(_item['family'] + ', ' + given + ', ')
    return ''.join(name[0:-1]) + 'and ' + name[-1][0:-2]

In [9]:
url = 'https://orcid.org/0000-0002-8021-7263'
driver = webdriver.Firefox()
driver.get(url)
page_source = driver.page_source
driver.close()
soup = BeautifulSoup(page_source, 'html.parser')

In [46]:
doi_list = []
doi_re = re.compile('doi.org')
for doi in soup.find_all(string='DOI:'):
    a = doi.parent.parent.findChild('a', href = doi_re)
    href = a.get('href')
    doi_list.append(href)

In [73]:
session = CrossRefClient()
prefix = '<li>'
suffix = '</li>'
for doi in doi_list:
    out = session.doi2json(doi)
    authors = names(out)
    year = str(out['created']['date-parts'][0][0])
    title = out['title']
    journal_short = out['container-title-short']
    volume = str(out['volume'])
    if 'article-number' in out.keys():
        pages = str(out['article-number'])
    else:
        pages = out['page']
    print(prefix +
          authors + ' ' +
          year + '. ' +
          title + '. ' +
          journal_short + '. ' +
          volume + ':' +
          pages + '. ' +
          'doi:' + doi + '.' +
          suffix)

Uszczynska-Ratajczak, B., Sugunan, S., Kwiatkowska, M., Migdal, M., Carbonell-Sala, S., Sokol, A., Winata, C.L., and Chacinska, A. 2022. Profiling subcellular localization of nuclear-encoded mitochondrial gene products in zebrafish. Life Sci. Alliance. 6:e202201514. doi:https://doi.org/10.26508/lsa.202201514
Migdał, M., Arakawa, T., Takizawa, S., Furuno, M., Suzuki, H., Arner, E., Winata, C.L., and Kaczkowski, B. 2023. xcore: an R package for inference of gene expression regulators. BMC Bioinformatics. 24:14. doi:https://doi.org/10.1186/s12859-022-05084-0
Migdał, M., Tralle, E., Abu Nahia, K., Bugajski, Ł., Kędzierska, K.Z., Garbicz, F., Piwocka, K., Winata, C.L., and Pawlak, M. 2021. Multi-omics analyses of early liver injury reveals cell-type-specific transcriptional and epigenomic shift. BMC Genomics. 22:904. doi:https://doi.org/10.1186/s12864-021-08173-1
Abu Nahia, K., Migdał, M., Quinn, T.A., Poon, K., Łapiński, M., Sulej, A., Liu, J., Mondal, S.S., Pawlak, M., Bugajski, Ł., Piwoc

In [74]:
a1, a2, a3 = ['a']

ValueError: not enough values to unpack (expected 3, got 1)